In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib.ticker as mtick
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
with pd.io.stata.StataReader("../dta files/2012.dta", convert_categoricals=True) as rdr:
    cols = [c for c in rdr.variable_labels().keys() if c not in {"ISCO88", "SPISCO88"}]
    df_2012 = rdr.read(columns=cols)

with pd.io.stata.StataReader("../dta files/2002.dta", convert_categoricals=True) as rdr:
    cols = [c for c in rdr.variable_labels().keys() if c not in ["v241","v247"]]
    df_2002 = rdr.read(columns=cols)

df_2022 = pd.read_stata("../dta files/2022.dta")


In [3]:
df_2002_clean = pd.read_csv("../data/final_csv/2002_final.csv")
df_2012_clean = pd.read_csv("../data/final_csv/2012_final.csv")
df_2022_clean = pd.read_csv("../data/final_csv/2022_final.csv")